### 미세 튜닝과 모범 사례

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


### 사전 준비
  * data_01.json 파일을 준비한다. (로컬에서 파일 내용을 생성하여 업로드 하거나, 구글 코랩에서 생성한다.)

### 학습 내용
 * 데이터 준비 - json을 jsonl로 변환
 * 미세 튜닝 모델 만들기
 * 미세 튜닝 리스트 확인
 * 미세 튜닝 모델 분석 및 삭제하기

data_01.json 파일 내용

```
{"prompt":"When do I have to start the heater?", "completion":"Every day in the morning at 7AM. You should stop it at 2PM"}
{"prompt":"Where is the garage remote control?", "completion":"Next to the yellow door, on the key ring"}
{"prompt":"Is it necessary to program the scent diffuser every day?", "completion":"The scent diffuser is already programmed, you just need to recharge it when its battery is low"}

```

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.3 MB/s eta 0:00:00


* 아래 OPENAI_API_KEY는 자신의 API 키를 입력해 주세요.

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = "sk-*"

### 모델에 학습할 수 있는 데이터 형식으로 변환

In [ ]:
!openai tools fine_tunes.prepare_data -f data_01.json

Analyzing...

- Your JSON file appears to be in a JSONL format. Your file will be converted to JSONL format
- Your file contains 3 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `?`
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Necessary] Your forma

### OpenAI의 API를 사용하여 Fine-Tuning 작업을 수행

In [ ]:
!openai api fine_tunes.create -t "data_01_prepared.jsonl" -m curie

Upload progress: 100% 417/417 [00:00<00:00, 216kit/s]
Uploaded file from data_01_prepared.jsonl: file-mEv3OfkBk8xB5qGxA3HcJ9mS
Created fine-tune: ft-a78fvHKwU3jGRs68DAZnttk0
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-08-21 02:15:31] Created fine-tune: ft-a78fvHKwU3jGRs68DAZnttk0
[2023-08-21 02:15:39] Fine-tune costs $0.00
[2023-08-21 02:15:39] Fine-tune enqueued. Queue number: 0



### 미세 튜닝한 모델 리스트 확인

In [ ]:
!openai api fine_tunes.list

{
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-xUtRhEjomukImmbA9CHncdHr",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": 2,
        "prompt_loss_weight": 0.01,
        "classification_n_classes": 7,
        "learning_rate_multiplier": 0.1,
        "compute_classification_metrics": true
      },
      "organization_id": "org-ujuF1J8GkXR8ggNiF9BMBXJp",
      "model": "curie",
      "training_files": [
        {
          "object": "file",
          "id": "file-jOHIwpQxKW5Ah9ewQY7MXQxm",
          "purpose": "fine-tune",
          "filename": "drug_malady_data_prepared_train.jsonl",
          "bytes": 129849,
          "created_at": 1691396784,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [
        {
          "object": "file",
          "id": "file-QLnF23VLmQJJytWTWMkGg7Ks",
          "purpose": "fine-tune",
          "filename": "drug_malady_data_prepared_valid.js

### 미세 튜닝된 모델을 활용하여 응답 얻기

* "fine_tuned_model": "curie:ft-personal-2023-08-14-07-49-58"를 이용

In [ ]:
os.environ['FINE_TUNED_MODEL'] = "curie:ft-personal-2023-08-21-02-27-03"

* 명령문
```
openai api completions.create -m $FINE_TUNED_MODEL -p < YOUR_PROMPT >
```

In [ ]:
!export FINE_TUNED_MODEL="curie:ft-personal-2023-08-21-02-27-03"
!openai api completions.create -m $FINE_TUNED_MODEL -p "Please complete the following sentences:\n\n1. When do I have to start the heater?\n   Response:"

Please complete the following sentences:\n\n1. When do I have to start the heater?\n   Response: You need to start the heater as soon as you have burned some of the diesel

### 모델 분석하기

* "id": "ft-L9e0mwZlrjH0my94TnMannkP"

In [ ]:
!openai api fine_tunes.results -i ft-L9e0mwZlrjH0my94TnMannkP

step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/weighted_f1_score
1,66,2,0.2644104197988054,0.0,0.0,0.29036785178662583,0.0,0.0,,
2,100,4,0.4237088239192962,0.0,0.0,,,,,
3,134,6,0.31941595206037166,0.0,0.0,,,,,
4,184,8,0.17455283596995289,0.0,0.0,,,,,
5,250,10,0.11041732400888575,0.5,0.5,,,,,
6,300,12,0.13137133310859403,0.0,0.0,,,,,
7,334,14,0.1697558584436774,0.0,0.0,,,,,
8,384,16,0.10107799720950424,1.0,1.0,,,,,
9,418,18,0.099259533546865,1.0,1.0,0.09764699074284484,0.5,0.5,,
10,452,20,0.2994108198583126,0.0,0.0,,,,,
11,486,22,0.16414392367005348,0.0,0.0,,,,,
12,520,24,0.13671906247735022,0.0,0.0,,,,,
13,554,26,0.08762960009276866,0.5,0.5,,,,,
14,604,28,0.20037543970936286,0.5,0.5,,,,,
15,654,30,0.054908831045031546,1.0,1.0,,,,,
16,752,32,0.039133796884270856,0.5,0.5,,,,,
17,802,34,0.19167657037576039,0.0,0.0,0.0987755858

### 매개 변수를 사용하여 미세 튜닝한 모델에 접미사 추가

In [ ]:
!openai api fine_tunes.create -t data_01_prepared.jsonl -m curie --suffix "daniel_toto"

Found potentially duplicated files with name 'data_01_prepared.jsonl', purpose 'fine-tune' and size 417 bytes
file-mEv3OfkBk8xB5qGxA3HcJ9mS
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 417/417 [00:00<00:00, 211kit/s]
Uploaded file from data_01_prepared.jsonl: file-ZBK6BIqDt5PUA65AYcbie9TY
Created fine-tune: ft-68tLamGm7pmkFmnsrhY7mptd
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-08-21 02:37:47] Created fine-tune: ft-68tLamGm7pmkFmnsrhY7mptd

[2023-08-21 02:37:56] Fine-tune costs $0.00
[2023-08-21 02:37:57] Fine-tune enqueued. Queue number: 0
Y
[2023-08-21 02:37:59] Fine-tune started
Y
Y



In [ ]:
!openai api fine_tunes.list

{
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-xUtRhEjomukImmbA9CHncdHr",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": 2,
        "prompt_loss_weight": 0.01,
        "classification_n_classes": 7,
        "learning_rate_multiplier": 0.1,
        "compute_classification_metrics": true
      },
      "organization_id": "org-ujuF1J8GkXR8ggNiF9BMBXJp",
      "model": "curie",
      "training_files": [
        {
          "object": "file",
          "id": "file-jOHIwpQxKW5Ah9ewQY7MXQxm",
          "purpose": "fine-tune",
          "filename": "drug_malady_data_prepared_train.jsonl",
          "bytes": 129849,
          "created_at": 1691396784,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [
        {
          "object": "file",
          "id": "file-QLnF23VLmQJJytWTWMkGg7Ks",
          "purpose": "fine-tune",
          "filename": "drug_malady_data_prepared_valid.js

### 모델 삭제하기

In [ ]:
!openai api models.delete -i "curie:ft-personal-2023-08-15-04-18-29"

{
  "id": "curie:ft-personal-2023-08-15-04-18-29",
  "object": "model",
  "deleted": true
}


In [ ]:
!openai api models.delete -i "curie:ft-personal:daniel-toto-2023-08-15-04-24-23"

{
  "id": "curie:ft-personal:daniel-toto-2023-08-15-04-24-23",
  "object": "model",
  "deleted": true
}
